# Урок 7: Semi-Structured Pruning (N:M Sparsity)

Паттерны разреженности, аппаратно поддерживаемые GPU. Самый частый — 2:4 (из каждых 4-х весов ровно 2 нулевых).

## 1. Математическая теория

### 1.1. Ограничения паттерна
Данный метод обеспечивает высокую разреженность, но при этом гарантированную структуру, удобную для параллельных вычислений в Tensor Cores.

### 1.2. Методы из обзора:
*   **SparseGPT (для N:M):** Позволяет применять 2:4 паттерн к огромным моделям, учитывая матрицу корреляций активаций для выбора «лучших» 2-х весов в группе.
*   **Wanda (для N:M):** Использует оценку магнитуды и энергии активаций для выбора 2-х весов из 4-х без обращения Гессиана. Это значительно быстрее на больших весах.
*   **E-Sparse (Li et al., 2023b):** Исследует влияние N:M разреженности на энергоэффективность (Energy-Efficiency) и предлагает стратегии для мобильных LLM.

---

In [ ]:
import torch

def apply_2_4_mask_raw(w):
    # Выбираем 2 из 4 самых больших по модулю в каждой группе
    w_block = w.view(-1, 4)
    _, idx = torch.topk(w_block.abs(), k=2, dim=1)
    mask = torch.zeros_like(w_block)
    mask.scatter_(1, idx, 1.0)
    return (w_block * mask).view(w.shape)

print("nanoGPT Track: Реализована логика наложения 2:4 маски через тензорные индексы.")

## 2. Промышленная реализация: PyTorch Semi-Structured Sparse
Начиная с PyTorch 2.1, поддержка 2:4 встроена нативно и позволяет получать ускорение до 2х на видеокартах Ampere/Hopper.

In [ ]:
"""
from torch.sparse import to_sparse_semi_structured
linear_layer.weight.data = to_sparse_semi_structured(apply_2_4_mask_raw(linear_layer.weight.data))
"""
print("Llama Track: Нативная поддержка 2:4 в PyTorch — стандарт для Ampere GPU.")